### 專案名稱: 第四屆機器學習百日馬拉松
### 功能描述: 第51~53天作業(期中考)
### 版權所有: Dunk  
### 程式撰寫: Dunk  
### 撰寫日期：2020/04/27
### 改版日期:  
### 改版備註:

請至以下Kaggle競賽平台期中考連結參加考試，請按join competition加入測驗 

並將任一次提交完成畫面截圖，上傳到github並回到官網完成提交。(提交畫面截圖範例)

https://www.kaggle.com/t/4e455552ac9b4a088ccae16e27d22be8

### ANS:

#### 匯入所需套件

In [10]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

In [11]:
# 資料維度顯示
print(df_train.shape)
print(df_test.shape)

(113, 22)
(33, 21)


In [14]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data)
    
def na_df_get(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    return missing_data

In [52]:
#顯示訓練集其中資料遺失比例
na_check(df_train)
df_na_train = na_df_get(df_train)
for i in range(df_na_train.shape[0]-1):
    df_train[df_na_train.index[i]].describe()
# df_na_train.index[df_na_train.shape[0]-1]

,Missing Ratio
loan_advances,98.230088
restricted_stock_deferred,91.150442
director_fees,88.495575
deferral_payments,75.221239
deferred_income,69.911504
long_term_incentive,56.637168
bonus,46.017699
from_messages,42.477876
from_poi_to_this_person,42.477876
from_this_person_to_poi,42.477876
